In [1]:
import library as l

import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import datetime
import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
'''#loadstring = 'seed_1519_size_20000_topsize_300_date_2018-04-12'
loadstring = 'seed_39169_size_40000_topsize_300_date_2018-04-12'

models = {}
for i, model in enumerate(['/LR_' + str(2001+x) + '.data' for x in range(14)]):
    models[i+1] = l.load_file(loadstring + model)

top_f = l.load_file(loadstring + '/top_f.data')
bot_f = l.load_file(loadstring + '/top_f.data')
CV = l.load_file(loadstring + '/CV_2001.data')
indices = l.load_file(loadstring + '/indices.data')
ratings = l.load_file(loadstring + '/ratings.data')
seen = l.load_file(loadstring + '/seen.data')
test_idx = l.load_file(loadstring + '/test_idx.data')
train_idx = l.load_file(loadstring + '/train_idx.data')
features = list(CV.get_feature_names())
print(len(features))'''

loadstring = []
loadstring.append('seed_6676_size_40000_topsize_300_date_2018-04-23')
loadstring.append('seed_15320_size_40000_topsize_300_date_2018-05-04')
loadstring.append('seed_39729_size_40000_topsize_300_date_2018-05-04')

loop = range(len(loadstring))

start_year = 2001
end_year = 2014

main = {}

for j, lstr in enumerate(loadstring):
    main[j] = {}

    main[j]['models'] = {}
    for i, model in enumerate(['/LR_' + str(start_year+x) + '.data' for x in range(end_year - start_year + 1)]):
        main[j]['models'][i+1] = l.load_file(lstr + model)

    main[j]['test_idx'] = {}
    for i, indx in enumerate(['/test_idx_' + str(start_year+x) + '.data' for x in range(end_year - start_year + 1)]):
        main[j]['test_idx'][i+1] = l.load_file(lstr + indx)

    main[j]['train_idx'] = {}
    for i, indx in enumerate(['/train_idx_' + str(start_year+x) + '.data' for x in range(end_year - start_year + 1)]):
        main[j]['train_idx'][i+1] = l.load_file(lstr + indx)

    main[j]['top_f'] = l.load_file(lstr + '/top_f.data')
    main[j]['bot_f'] = l.load_file(lstr + '/top_f.data')
    main[j]['CV'] = l.load_file(lstr + '/CV_2001.data')
    main[j]['indices'] = l.load_file(lstr + '/indices.data')
    main[j]['ratings'] = l.load_file(lstr + '/ratings.data')
    main[j]['seen'] = l.load_file(lstr + '/seen.data')
    main[j]['features'] = list(main[j]['CV'].get_feature_names())

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
from collections import defaultdict
import json

def load_previous_data(indices, file='aggressive_dedup.json'):
    data = defaultdict(list)
    time = defaultdict(list)
    #ratings = defaultdict(list)
    
    idx = sorted([item for sublist in indices.values() for item in sublist])
    x = sorted(list(indices.keys()))
    for yr in x:
        data[yr] = list(np.zeros(len(indices[yr])))
        time[yr] = list(np.zeros(len(indices[yr])))
        #ratings[yr] = list(np.zeros(len(indices[yr])))
    with open(file) as infile:
        i = 0
        for line in infile:
            if i == idx[0]:
                idx.pop(0)
                x = json.loads(line)
                yr = x['reviewTime'][-4:]
                j = indices[yr].index(i)
                data[yr][j] = x.get('reviewText')
                time[yr][j] = x.get('reviewTime')
                #ratings[yr][j] = int(x.get('overall'))    
            i += 1
            if len(idx) == 0:
                break
    
    return data, time#, ratings

In [4]:
for z in loop:
    main[z]['data'], main[z]['time'] = load_previous_data(main[z]['indices'])
    #l.simplify_ratings(main[z]['data'], main[z]['ratings'])

In [5]:
main[z]['data'].keys()

dict_keys(['2010', '2003', '2007', '2014', '2008', '2004', '2001', '2011', '2005', '2009', '2006', '2013', '2002', '2012'])

In [6]:
main[z]['ratings'].keys()

dict_keys(['2010', '2003', '2013', '2014', '2008', '2004', '2001', '2011', '2005', '2009', '2006', '2007', '2002', '2012'])

In [7]:
x = sorted(main[0]['models'].keys())

results = []


for i, each in enumerate(x):
    tempo = []
    for z in loop:
        temp = []
        #arr = np.array(main[z]['data'][str(2000 + each)])
        #X_train, X_test = arr[main[z]['train_idx'][each]], arr[main[z]['test_idx'][each]]
        
        X_test = np.array(main[z]['data'][str(2000 + each)])[main[z]['test_idx'][each]]
        
        #arr = np.array(main[z]['ratings'][str(2000 + each)])
        #y_train, y_test = arr[main[z]['train_idx'][each]], arr[main[z]['test_idx'][each]]
        
        #X_train = main[z]['CV'].transform(X_train)
        
        X_test = main[z]['CV'].transform(X_test)
        y_test = np.array(main[z]['ratings'][str(2000 + each)])[main[z]['test_idx'][each]]
        
        temp.append(main[z]['models'][each].score(X_test, y_test))

        print('Trained on: ' + str(2000 + each) + '  Scored on: ' + str(2000 + each) + '    acc: ', temp[-1])
        
        for every in x[i+1:]:
            #arr = np.array(main[z]['data'][str(2000 + every)])
            #X_train, X_test = arr[main[z]['train_idx'][every]], arr[main[z]['test_idx'][every]]
            
            X_test = np.array(main[z]['data'][str(2000 + every)])[main[z]['test_idx'][every]]
            
            #arr = np.array(main[z]['ratings'][str(2000 + every)])
            #y_train, y_test = arr[main[z]['train_idx'][every]], arr[main[z]['test_idx'][every]]

            #X_train = main[z]['CV'].transform(X_train)
            X_test = main[z]['CV'].transform(X_test)
            y_test = np.array(main[z]['ratings'][str(2000 + every)])[main[z]['test_idx'][every]]
            
            temp.append(main[z]['models'][each].score(X_test, y_test))
            
            print('Trained on: ' + str(2000 + each) + '  Scored on: ' + str(2000 + every) + '    acc: ', temp[-1])
        print()
        tempo.append(temp)
    results.append(tempo)
    if i == len(x) - 2:
        break

Trained on: 2001  Scored on: 2001    acc:  0.879625
Trained on: 2001  Scored on: 2002    acc:  0.875375
Trained on: 2001  Scored on: 2003    acc:  0.863375
Trained on: 2001  Scored on: 2004    acc:  0.855125
Trained on: 2001  Scored on: 2005    acc:  0.854875
Trained on: 2001  Scored on: 2006    acc:  0.862625
Trained on: 2001  Scored on: 2007    acc:  0.859
Trained on: 2001  Scored on: 2008    acc:  0.8485
Trained on: 2001  Scored on: 2009    acc:  0.85375
Trained on: 2001  Scored on: 2010    acc:  0.843875
Trained on: 2001  Scored on: 2011    acc:  0.834875
Trained on: 2001  Scored on: 2012    acc:  0.861125
Trained on: 2001  Scored on: 2013    acc:  0.858125
Trained on: 2001  Scored on: 2014    acc:  0.85075

Trained on: 2001  Scored on: 2001    acc:  0.8785
Trained on: 2001  Scored on: 2002    acc:  0.8775
Trained on: 2001  Scored on: 2003    acc:  0.86625
Trained on: 2001  Scored on: 2004    acc:  0.854875
Trained on: 2001  Scored on: 2005    acc:  0.856375
Trained on: 2001  Score

In [12]:
#l.save_file('run_once_results.data', results)

In [13]:
results = l.load_file('run_once_results.data')

In [14]:
results

[[[0.87962499999999999,
   0.87537500000000001,
   0.863375,
   0.85512500000000002,
   0.85487500000000005,
   0.86262499999999998,
   0.85899999999999999,
   0.84850000000000003,
   0.85375000000000001,
   0.84387500000000004,
   0.83487500000000003,
   0.86112500000000003,
   0.85812500000000003,
   0.85075000000000001],
  [0.87849999999999995,
   0.87749999999999995,
   0.86624999999999996,
   0.85487500000000005,
   0.856375,
   0.85450000000000004,
   0.85612500000000002,
   0.84987500000000005,
   0.85212500000000002,
   0.84924999999999995,
   0.84325000000000006,
   0.86024999999999996,
   0.86012500000000003,
   0.85424999999999995],
  [0.87312500000000004,
   0.872,
   0.864375,
   0.85275000000000001,
   0.84824999999999995,
   0.85699999999999998,
   0.86575000000000002,
   0.85350000000000004,
   0.84937499999999999,
   0.85050000000000003,
   0.84162499999999996,
   0.85137499999999999,
   0.85487500000000005,
   0.84524999999999995]],
 [[0.86762499999999998,
   0.862125